In [0]:
#Download Fasttext embeddings
!wget https://s3-us-west-1.amazonaws.com/fasttext-vectors/crawl-300d-2M-subword.zip

--2018-11-22 06:06:36--  https://s3-us-west-1.amazonaws.com/fasttext-vectors/crawl-300d-2M-subword.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.116.9
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.116.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5828358084 (5.4G) [application/zip]
Saving to: ‘crawl-300d-2M-subword.zip’

crawl-300d-2M-subwo 100%[===================>]   5.43G  25.1MB/s    in 2m 41s  

2018-11-22 06:09:17 (34.5 MB/s) - ‘crawl-300d-2M-subword.zip’ saved [5828358084/5828358084]



In [0]:
!unzip crawl-300d-2M-subword.zip

Archive:  crawl-300d-2M-subword.zip
  inflating: crawl-300d-2M-subword.vec  
  inflating: crawl-300d-2M-subword.bin  


In [0]:
!rm crawl-300d-2M-subword.zip glove.840B.300d.txt crawl-300d-2M-subword.bin glove_weights_base.best.hdf5 biGRU_Glove_840B.csv

In [0]:
!ls -lrt

total 4546472
-rw-r--r-- 1 root root 4520128711 Aug 22 20:30 crawl-300d-2M-subword.vec
drwxr-xr-x 2 root root       4096 Nov 20 18:17 sample_data
-rw-r--r-- 1 root root   60354593 Nov 22 04:21 test.csv
-rw-r--r-- 1 root root   68802655 Nov 22 04:21 train.csv
-rw-r--r-- 1 root root    6279782 Nov 22 04:21 sample_submission.csv


In [0]:
!pip install googledrivedownloader

In [0]:
from google_drive_downloader import GoogleDriveDownloader as gdd

ids =["1EO59IOOCVcqKymPzTOVHKxTmIjEKEfW2", "1pSY29XeBuodiIZdtGDtq52MZsAJTAKdo", "1Jt7Wr3RUQRnLD0AuirxPCxjkljfumPLw"]

file_names = ["test.csv","train.csv","sample_submission.csv"]
for i in range(len(ids)):
  a = gdd.download_file_from_google_drive(file_id= ids[i],
                                      dest_path="./"+file_names[i],
                                      unzip=False)

In [0]:
import pandas as pd
import numpy as np

from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

Using TensorFlow backend.


In [0]:
train = pd.read_csv("train.csv", header =0)
test = pd.read_csv("test.csv", header = 0)

In [0]:
train["comment_text"].fillna("dummytext")
test["comment_text"].fillna("dummytext")
X_train = train["comment_text"].str.lower()
X_test = test["comment_text"].str.lower()
Y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

In [0]:
#Parameters
embedding_size = 300
maxlen = 150
max_vocab_size = 100000

In [0]:
tokenizer = text.Tokenizer(num_words = max_vocab_size, lower = False)  
# num_words - will consider only top max_vocab_size words based on frequency count
# So, when we do texts_to_sequences, the max value will be 100,000. 
# But, tokinzer on its own will contain word and indices for all the words in the corpus.

tokenizer.fit_on_texts(list(X_train) + list(X_test))
#Its not necessary to fit on both the train and test. If we do that, we take into account the top words of test set as well 

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [0]:
glove_embeddings = {}

with open("crawl-300d-2M-subword.vec", encoding= "utf8" ) as f:
  for line in f:
    words_vector = line.rstrip().rsplit(" ")
    word = words_vector[0]
    vector = np.asarray(words_vector[1:], dtype= 'float32')
    glove_embeddings[word] = vector

print("There are: "+str(len(glove_embeddings))+" fasttext word embeddings")

There are: 2000000 fasttext word embeddings


In [0]:
max_word_index_tokenizer = len(tokenizer.word_index)
num_words = min(max_vocab_size, max_word_index_tokenizer)

In [0]:
oov_count = 0
embedding_matrix = np.ones((num_words, embedding_size))
for word, i in tokenizer.word_index.items():
  if i>= max_vocab_size:
    continue
  embedding_vector = glove_embeddings.get(word)
  
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector 
   
  else:
    oov_count+=1
   
  
print("There are "+str(oov_count)+" missing fasttext word embeddings") 
    
  

There are 25989 missing fasttext word embeddings


In [0]:
sequence_input = Input(shape = (maxlen,)) # Every senetence will have a maxlength of 150.
x = Embedding(input_dim = max_vocab_size, output_dim = embedding_size, trainable = False, weights = [embedding_matrix]) (sequence_input)
x = Bidirectional(GRU(128, return_sequences=True)) (x)
# 128 is the number of hidden units in each GRU cell. 
# return_sequences makes the output of each GRU cell as the input to next cell. 
# We have 128 neurons in each GRU cell. Since it is bidirectional GRU, the forward and the backward pass
# are concatenated and we get 256 neuron ouput from each GRU cell.
# Since we have 150 as max sequence length, the dimension after this cell is (150, 256)

x = GlobalMaxPooling1D()(x)
predictions = Dense(6, activation="sigmoid")(x)

model = Model(sequence_input, predictions)
model.summary()
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 300)          30000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 150, 256)          329472    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 1542      
Total params: 30,331,014
Trainable params: 331,014
Non-trainable params: 30,000,000
_________________________________________________________________


In [0]:
batch_size = 128
epochs = 4
X_train_sub, X_val_sub, Y_train_sub, Y_val_sub = train_test_split(X_train, Y_train, train_size=0.9, random_state=123)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [0]:
file_path="glove_weights_base.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

early = EarlyStopping(monitor="val_loss", mode="min", patience=5)


callbacks_list = [checkpoint, early] #early
model.fit(X_train_sub, Y_train_sub, batch_size=batch_size, epochs=epochs, validation_data=(X_val_sub, Y_val_sub), callbacks=callbacks_list)

Train on 143613 samples, validate on 15958 samples
Epoch 1/4
143613/143613 [==============================] - 801s 6ms/step - loss: 0.0712 - acc: 0.9759 - val_loss: 0.0470 - val_acc: 0.9822

Epoch 00001: val_loss improved from inf to 0.04702, saving model to glove_weights_base.best.hdf5
Epoch 2/4
143613/143613 [==============================] - 788s 5ms/step - loss: 0.0496 - acc: 0.9814 - val_loss: 0.0425 - val_acc: 0.9836

Epoch 00002: val_loss improved from 0.04702 to 0.04248, saving model to glove_weights_base.best.hdf5
Epoch 3/4
143613/143613 [==============================] - 785s 5ms/step - loss: 0.0449 - acc: 0.9829 - val_loss: 0.0422 - val_acc: 0.9838

Epoch 00003: val_loss improved from 0.04248 to 0.04218, saving model to glove_weights_base.best.hdf5
Epoch 4/4
143613/143613 [==============================] - 783s 5ms/step - loss: 0.0423 - acc: 0.9836 - val_loss: 0.0392 - val_acc: 0.9850

Epoch 00004: val_loss improved from 0.04218 to 0.03924, saving model to glove_weights_base

In [0]:
model.load_weights(file_path)
test_predictions = model.predict(X_test,batch_size=1024,verbose=1)

153164/153164 [==============================] - 47s 309us/step


In [0]:
submission = pd.read_csv('sample_submission.csv')
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = test_predictions
submission.to_csv('biGRU_fasttext.csv', index=False)

In [0]:
from google.colab import files

files.download("biGRU_fasttext.csv") 

In [0]:
#0.9826 Private  0.9843 Public